In [ ]:
#| label: arrplot

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, FloatText, IntText
from matplotlib.patches import Circle
import pandas as pd
import scipy.io
import ipywidgets as widgets
from ipywidgets import FloatText, FloatSlider, interact

# Load the mat file with coordinates
mat = scipy.io.loadmat('./dataAbstract/mapEUROPE.mat')
MAPeu = mat['MAPeu']

# Load coordinate data
df = pd.read_csv("./dataAbstract/out1costo.txt",
                 header=None,
                 sep=r"\s+")

c1 = df[0].values
c2 = df[1].values

x_coords = c1
y_coords = c2

# Load the new MATLAB file with 5 variables
data_input = scipy.io.loadmat('./dataAbstract/datain100_v7.mat')

# Check what variables are in the .mat file
#print("Variables in datainput100.mat:", list(data_input.keys()))

# Extract the 5 variables (adjust variable names as needed)
var_names = []
for key in data_input.keys():
    if not key.startswith('__'):  # Skip MATLAB metadata
        if key != 'MAPeu':  # Skip if it's the map variable
            var_names.append(key)

# Take the first 5 variables (or adjust as needed)
var_names = [var_names[i] for i in [0, 4, 1, 2, 3]]
var_names = var_names[:5]
#print(f"Using variables: {var_names}")

# Create Z dictionary with the 5 variables
Z_vars = {}
for i, var_name in enumerate(var_names):
    var_data = data_input[var_name]
    # If the variable is 2D (n x m), we'll use it as is
    # If it's 1D, we'll reshape it to 2D with m=1
    if len(var_data.shape) == 1:
        var_data = var_data.reshape(-1, 1)
    Z_vars[f'Z{i+1}'] = var_data

# Store the number of points (n) and variables per point (m)
n_points = len(x_coords)
m_values = Z_vars['Z1'].shape[1]  # Number of variables per point
z_key2  = [r'Economic losses (€ $\mathrm{ha}^{-1}\,\mathrm{yr}^{-1})$', r'Yield losses (Mg $\mathrm{ha}^{-1}\,\mathrm{yr}^{-1})$',
           r'C losses (Mg $\mathrm{ha}^{-1}\,\mathrm{yr}^{-1})$',
           r'$\mathrm{N}_{2}O$ losses (kg $\mathrm{ha}^{-1}\,\mathrm{yr}^{-1})$',
           r'$\mathrm{NO}_{3}$ losses (kg $\mathrm{ha}^{-1}\,\mathrm{yr}^{-1})$']

#print(f"Number of points (n): {n_points}")
#print(f"Variables per point (m): {m_values}")

def plot_colormap_with_histograms(alpha, vmin, vmax, center_x, center_y):
    """Plot main colormap on left and histograms for the nearest point on right"""

    # Find the nearest point to the selected coordinates
    distances = np.sqrt((x_coords - center_x)**2 + (y_coords - center_y)**2)
    point_index = np.argmin(distances)
    min_distance = distances[point_index]

    # Get the selected Z variable for colormap
    Z_selected = Z_vars['Z1']

    # For colormap, use the mean of m values or first value
    Z_mean_values = Z_selected.mean(axis=1) if Z_selected.shape[1] > 1 else Z_selected.flatten()
    Z_scaled = alpha * Z_mean_values

    # Get values at the nearest point (already multiplied by alpha)
    nearest_point_values = {}
    for key in Z_vars.keys():
        nearest_point_values[key] = Z_vars[key][point_index] * alpha/100

    # Create figure with custom layout: main panel on left, special layout on right
    fig = plt.figure(figsize=(20, 10))

    # Create gridspec: 1 row, 2 columns (left: main, right: special layout)
    gs = fig.add_gridspec(1, 2, width_ratios=[2, 1], wspace=0.15)

    # Left panel: main colormap
    ax_main = fig.add_subplot(gs[0])

    # Right panel: create special layout (1, 2, 2)
    gs_right = gs[1].subgridspec(3, 2, hspace=0.35, wspace=0.25, height_ratios=[2, 1, 1])

    # Create axes according to (1, 2, 2) layout
    # Top: single histogram spanning 2 columns
    ax_hist_top = fig.add_subplot(gs_right[0, :])

    # Middle: two histograms
    ax_hist_mid1 = fig.add_subplot(gs_right[1, 0])
    ax_hist_mid2 = fig.add_subplot(gs_right[1, 1])

    # Bottom: two histograms
    ax_hist_bot1 = fig.add_subplot(gs_right[2, 0])
    ax_hist_bot2 = fig.add_subplot(gs_right[2, 1])

    # Store histogram axes in order
    hist_axes = [ax_hist_top, ax_hist_mid1, ax_hist_mid2, ax_hist_bot1, ax_hist_bot2]

    # Get variable names for histograms
    z_keys = list(Z_vars.keys())

    # Plot 1: Main colormap (LEFT SIDE)
    scatter = ax_main.scatter(x_coords, y_coords, c=Z_scaled, cmap='YlGnBu',
                              s=2, alpha=0.8, vmin=vmin, vmax=vmax)

    # Highlight ONLY the nearest point
    nearest_value = Z_scaled[point_index]
    ax_main.scatter(x_coords[point_index], y_coords[point_index],
                   c='red', s=140, marker='o', edgecolors='red', linewidth=3,
                   label=(
                   f"Location (lat, lon):\n"
                   f"({x_coords[point_index]:.1f}, {y_coords[point_index]:.1f})\n"
                   f"Cost: {nearest_value:.0f} € "
                   r"$\mathrm{ha}^{-1}\,\mathrm{yr}^{-1}$"))

    # Plot map contours
    for vcon_cell in MAPeu.flatten():
        vcon = np.array(vcon_cell)
        ax_main.plot(vcon[:,0], vcon[:,1], color='k', linewidth=2.5, alpha=0.7)

    ax_main.set_xlabel('Longitude', fontsize=19)
    ax_main.set_ylabel('Latitude', fontsize=19)
    ax_main.legend(loc='upper left', fontsize=19)
    ax_main.set_aspect('equal', adjustable='box')
    ax_main.grid(True, alpha=0.3)
    ax_main.set_xlim(-12, 40)
    ax_main.set_ylim(33, 73)
    ax_main.tick_params(axis='both', labelsize=19)

    # Add colorbar for main plot
    cbar = fig.colorbar(scatter, ax=ax_main, shrink=0.8, pad=0.02)
    cbar.ax.tick_params(labelsize=19)
    cbar.set_label(r'Economic losses (€ $\mathrm{ha}^{-1}\,\mathrm{yr}^{-1})$', fontsize=19)

    # Define colors for histograms
    colors = ['skyblue', 'lightgreen', 'salmon', 'gold', 'violet']

    # Plot histograms for the nearest point (RIGHT SIDE, special layout)
    for i, (ax, z_key, color) in enumerate(zip(hist_axes, z_keys, colors)):
        # Get the m values for this variable at the nearest point (already multiplied by alpha)
        point_data = nearest_point_values[z_key]

        # Plot histogram of the m values
        if len(point_data) > 0:
            # Calculate optimal number of bins (more for top histogram)
            if i == 0:  # Top histogram (larger)
                n_bins = min(30, max(10, int(len(point_data) / 3)))
            else:  # Smaller histograms
                n_bins = min(15, max(5, int(len(point_data) / 5)))

            # Plot histogram
            n, bins, patches = ax.hist(point_data, bins=n_bins, edgecolor='black',
                                      alpha=0.7, color=color, density=False)

            # Add vertical line for mean value
            mean_val = np.mean(point_data)
            ax.axvline(x=mean_val, color='red', linestyle='--', linewidth=2)
            ax.tick_params(axis='both', labelsize=15)
            ax.xaxis.label.set_size(15)
            ax.yaxis.label.set_size(15)

            # Add statistics text box
            #stats_text = (f'Mean = {mean_val:.3f}\n'
            #             f'Std = {np.std(point_data):.3f}')

            # Position text - upper right for all except top (which uses upper left)
            #if i == 0:  # Top histogram
            #    ax.text(0.05, 0.95, stats_text, transform=ax.transAxes,
            #            verticalalignment='top', horizontalalignment='left',
            #            fontsize=17, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
            #else:  # Smaller histograms
            #    ax.text(0.95, 0.95, stats_text, transform=ax.transAxes,
            #            verticalalignment='top', horizontalalignment='right',
            #            fontsize=14, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

            ax.set_xlabel(f'{z_key2[i]}')
            if i == 0 or i == 1 or i == 3:  # Label y-axis for left column histograms
                ax.set_ylabel('Frequency')


            ax.grid(True, alpha=0.3, linestyle='--')
            #ax.legend(fontsize=18, loc='best' if i == 0 else 'best')

            # Adjust x-axis limits to show data better
            data_range = np.ptp(point_data)
            if data_range > 0:
                padding = 0.1 * data_range
                ax.set_xlim(np.min(point_data) - padding,
                           np.max(point_data) + padding)
            else:
                # If all values are the same, add some padding
                ax.set_xlim(point_data[0] - 1, point_data[0] + 1)
        else:
            ax.text(0.5, 0.5, f'No data\nfor {z_key}',
                    transform=ax.transAxes,
                    horizontalalignment='center',
                    verticalalignment='center',
                    fontsize=18)
            ax.set_xlabel(f'{z_key2[i]}')
            if i == 0 or i == 1 or i == 3:
                ax.set_ylabel('Frequency')


    # Adjust layout
    fig.subplots_adjust(
        left=0.05,
        right=0.95,
        bottom=0.06,
        top=0.92,
        wspace=0.15,
        hspace=0.35
    )
    plt.show()

# Create interactive widget
alpha = FloatText(value=20, description='%Compacted')

vmin = FloatSlider(
    min=-25, max=300, step=0.1, value=0,
    description='min €'
)

vmax = FloatSlider(
    min=-25, max=300, step=0.1, value=50,
    description='max €'
)

center_x = FloatSlider(
    min=-12, max=40, step=0.5, value=6.49,
    description='Longitude'
)

center_y = FloatSlider(
    min=33, max=73, step=0.5, value=51.069861,
    description='Latitude'
)

# Two columns
left_col = widgets.VBox([alpha, vmin, vmax])
right_col = widgets.VBox([center_x, center_y])

ui = widgets.HBox([left_col, right_col])

out = widgets.interactive_output(
    plot_colormap_with_histograms,
    {
        'alpha': alpha,
        'vmin': vmin,
        'vmax': vmax,
        'center_x': center_x,
        'center_y': center_y
    }
)

display(ui, out)

Output()

# Nouvelle section